In [13]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re

# Load models and tokenizers
try:
    binary_model = AutoModelForSequenceClassification.from_pretrained("binary_model/checkpoint-2253")
    multilabel_model = AutoModelForSequenceClassification.from_pretrained("multilabel_model/checkpoint-2253")
    tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")
except Exception as e:
    messagebox.showerror("Error", f"Failed to load models or tokenizer: {e}")
    raise SystemExit(f"Failed to load models or tokenizer: {e}")

# Load persuasion techniques from a file
with open('persuasion_techniques_list.txt', 'r') as file:
    persuasion_techniques = [line.strip() for line in file.readlines()]


# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
binary_model.to(device)
multilabel_model.to(device)

# Function to check if the text is in Arabic
def is_arabic(text):
    return re.search(r'[\u0600-\u06FF]', text) is not None

# Define the function to make predictions
def predict(text, model_type):
    try:
        # Check if the input is empty
        if not text.strip():
            messagebox.showerror("Input Error", "Please enter some text.")
            return
        
        # Check if the text is in Arabic
        if not is_arabic(text):
            messagebox.showerror("Input Error", "Please enter text in Arabic.")
            return
        
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        if model_type == "binary":
            with torch.no_grad():
                outputs = binary_model(**inputs)
            label = torch.argmax(outputs.logits, dim=1).item()
            result = "Propaganda" if label == 1 else "Not Propaganda"
        else:
            with torch.no_grad():
                outputs = multilabel_model(**inputs)
            sigmoid_outputs = torch.sigmoid(outputs.logits).cpu().numpy()[0]
            
            # Define a threshold to consider as positive (e.g., 0.2)
            threshold = 0.2
            predicted_labels = [persuasion_techniques[i] for i in range(len(sigmoid_outputs)) if sigmoid_outputs[i] >= threshold]
            
            if "no_technique" in predicted_labels:
                if len(predicted_labels) == 1:  # Only "no technique" detected
                    result = "No Technique detected."
                else:
                    predicted_labels.remove("no_technique")
                    result = ", ".join(predicted_labels)
            else:
                result = ", ".join(predicted_labels) if predicted_labels else "No persuasion techniques detected."
        
        result_label.config(text=f"Result: {result}")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred during prediction: {e}")

def analyze_file():
    try:
        file_path = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
        if not file_path:
            return  # User cancelled the file dialog
        
        with open(file_path, 'r') as file:
            text = file.read()
        
        predict(text, model_type_var.get())
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred while analyzing the file: {e}")

# Initialize the main application window
root = tk.Tk()
root.title("Propaganda Detection")

# Apply a style for the ttk widgets
style = ttk.Style(root)
style.configure("TButton", font=("Helvetica", 10))
style.configure("TLabel", font=("Helvetica", 12))
style.configure("TCombobox", font=("Helvetica", 10))
style.configure("TFrame", padding=10)

# Create and grid the main frame
main_frame = ttk.Frame(root)
main_frame.grid(padx=20, pady=20)

# Header
header_label = ttk.Label(main_frame, text="Propaganda Detection Tool", font=("Helvetica", 16, "bold"))
header_label.grid(column=0, row=0, columnspan=2, pady=10)

# Dropdown for selecting the model type
model_type_label = ttk.Label(main_frame, text="Select Model Type:")
model_type_label.grid(column=0, row=1, sticky="W", pady=5)
model_type_var = tk.StringVar(value="binary")
model_selection = ttk.Combobox(main_frame, textvariable=model_type_var, state="readonly")
model_selection['values'] = ("binary", "multilabel")
model_selection.grid(column=1, row=1, pady=5, sticky="W")

# Textbox for paragraph input
input_label = ttk.Label(main_frame, text="Enter the Article:")
input_label.grid(column=0, row=2, sticky="W", pady=5)
text_input = tk.Text(main_frame, height=10, width=50, font=("Helvetica", 10))
text_input.grid(column=0, row=3, columnspan=2, pady=5)

# Button to make the prediction
predict_button = ttk.Button(main_frame, text="Predict", command=lambda: predict(text_input.get("1.0", tk.END), model_type_var.get()))
predict_button.grid(column=0, row=4, pady=10)

# Button to upload and analyze a file
file_button = ttk.Button(main_frame, text="Analyze File", command=analyze_file)
file_button.grid(column=1, row=4, pady=10)

# Label to show the result
result_label = ttk.Label(main_frame, text="Result: ", font=("Helvetica", 12, "bold"))
result_label.grid(column=0, row=5, columnspan=2, pady=10)

# Add padding to all children of the main_frame
for child in main_frame.winfo_children():
    child.grid_configure(padx=5, pady=5)

# Run the application
root.mainloop()
